In [1]:
# importing experiment data
import numpy as np
import pandas as pd
import ast
import math
import matplotlib.pyplot as plt
from scipy.stats import variation



In [ ]:
# To-Do
# visualize average cv changes across all IKI positions for each individual
# find average IKI values for each individual
#